In [1]:
import numpy as np
import pandas as pd
import os
from fancyimpute import KNN
from sklearn.model_selection import train_test_split
from xgboost import XGBRFClassifier
from sklearn.metrics import mean_absolute_error

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Using TensorFlow backend.


/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [6]:
train_data['IsMale'] = train_data['Sex'].apply(lambda x: 1 if x == 'male' else 0)
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,IsMale
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1


In [7]:
test_data['IsMale'] = test_data['Sex'].apply(lambda x: 1 if x == 'male' else 0)
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,IsMale
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,1
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,0


In [8]:
y = train_data["Survived"]

features = ["Pclass", "IsMale", "SibSp", "Parch", "Age", "Fare"]
X = train_data[features]
X.tail()

,Pclass,IsMale,SibSp,Parch,Age,Fare
886,2,1,0,0,27.0,13.00
887,1,0,0,0,19.0,30.00
888,3,0,1,2,NaN,23.45
889,1,1,0,0,26.0,30.00
890,3,1,0,0,32.0,7.75


In [9]:
X_filled = KNN(k=3).fit_transform(X)
X_filled[-5:]

Imputing row 1/891 with 0 missing, elapsed time: 0.147
Imputing row 101/891 with 0 missing, elapsed time: 0.149
Imputing row 201/891 with 0 missing, elapsed time: 0.151
Imputing row 301/891 with 1 missing, elapsed time: 0.153
Imputing row 401/891 with 0 missing, elapsed time: 0.154
Imputing row 501/891 with 0 missing, elapsed time: 0.156
Imputing row 601/891 with 0 missing, elapsed time: 0.157
Imputing row 701/891 with 0 missing, elapsed time: 0.159
Imputing row 801/891 with 0 missing, elapsed time: 0.160


array([[ 2.        ,  1.        ,  0.        ,  0.        , 27.        ,
        13.        ],
       [ 1.        ,  0.        ,  0.        ,  0.        , 19.        ,
        30.        ],
       [ 3.        ,  0.        ,  1.        ,  2.        , 28.59372303,
        23.45      ],
       [ 1.        ,  1.        ,  0.        ,  0.        , 26.        ,
        30.        ],
       [ 3.        ,  1.        ,  0.        ,  0.        , 32.        ,
         7.75      ]])

In [10]:
X_filled = pd.DataFrame(data=X_filled,columns=['Pclass','IsMale','SibSp','Parch','Age','Fare'])
X_filled.head()

,Pclass,IsMale,SibSp,Parch,Age,Fare
0,3.0,1.0,1.0,0.0,22.0,7.2500
1,1.0,0.0,1.0,0.0,38.0,71.2833
2,3.0,0.0,0.0,0.0,26.0,7.9250
3,1.0,0.0,1.0,0.0,35.0,53.1000
4,3.0,1.0,0.0,0.0,35.0,8.0500


In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(X_filled, y, random_state=1)

model = XGBRFClassifier(n_estimators=1000, max_depth=5, random_state=1, learning_rate=0.5)
model.fit(X_train, y_train,
         early_stopping_rounds=5,
         eval_set=[(X_valid, y_valid)],
         verbose=False)
predictions = model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

Mean Absolute Error: 0.21076233183856502


In [12]:
X_test = test_data[features]
X_test_filled = KNN(k=3).fit_transform(X_test)
X_test_filled = pd.DataFrame(data=X_test_filled,columns=['Pclass','IsMale','SibSp','Parch','Age','Fare'])
X_test_filled

Imputing row 1/418 with 0 missing, elapsed time: 0.035
Imputing row 101/418 with 0 missing, elapsed time: 0.036
Imputing row 201/418 with 1 missing, elapsed time: 0.037
Imputing row 301/418 with 0 missing, elapsed time: 0.039
Imputing row 401/418 with 0 missing, elapsed time: 0.040


,Pclass,IsMale,SibSp,Parch,Age,Fare
0,3.0,1.0,0.0,0.0,34.500000,7.8292
1,3.0,0.0,1.0,0.0,47.000000,7.0000
2,2.0,1.0,0.0,0.0,62.000000,9.6875
3,3.0,1.0,0.0,0.0,27.000000,8.6625
4,3.0,0.0,1.0,1.0,22.000000,12.2875
...,...,...,...,...,...,...
413,3.0,1.0,0.0,0.0,25.000000,8.0500
414,1.0,0.0,0.0,0.0,39.000000,108.9000
415,3.0,1.0,0.0,0.0,38.500000,7.2500
416,3.0,1.0,0.0,0.0,25.000000,8.0500


In [13]:
predictions_final = model.predict(X_test_filled)
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions_final})
output

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [14]:
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
